# MUSICAL PATTERNS

An introduction to musical pattern generation in musx. This tutoral starts with easier patterns and proceeds to more involved ones over the course of the notebook. The first pattern (Cycle) also  introduces the general features of all musx patterns so if you are new to musx that is the place to start. The musx pattern documentation and demos also provide information and examples.

To jump to a specific pattern inside this notebook use these links:

#### | [Cycle](#cycle_section) | [Palindrome](#palindrome_section) | [Range](#range_section) | [Shuffle](#shuffle_section) | [Choose](#choose_section) | [Graph](#graph_section) | [Rotation](#rotation_section) | [Markov](#markov_section) | [States](#states_section) |

<!--
- [Cycle](#cycle_section)
- [Palindrome](#palindrome_section)
- [Range](#range_section)
- [Shuffle](#shuffle_section)
- [Choose](#choose_section)
- [Graph](#graph_section)
- [Rotation](#rotation_section)
- [Markov](#markov_section)
- [States](#states_section)
-->

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from random import randint
from musx import Cycle, Palindrome, Range, Choose, Shuffle, Graph, Rotation, Markov, States,\
version, setmidiplayer, playfile, Score, Note, Seq, MidiFile, keynum, between, interp,\
intempo, pick, rescale, odds, fit
from musx.midi.gm import ChoirAahs, AcousticGrandPiano, Marimba
print(f"musx version: {version}")

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

<!-- [The Choose pattern](#The-Choose-pattern) -->

## Musical patterns in musx

Music is brimming with patterns! Composers are always looking for new ways to organize notes, rhythms, even elements of form. By incorporating patterns at various levels of their music composers can control the "mix" of new and heard-before ideas in artistic and interesting ways. 

In terms of coding, a pattern is a resource that generates data in a particular ordering. Since there are so many kinds of patterns in music, it is useful to have a toolbox of generic patterns and a way to implement new ones.

In musx patterns are subclasses of a Pattern base class that implements an object-orientated interface for generating data, from simple looping and randomness to more complex processes such as Markov chains, cellular automata and chaos.  
Many of these patterns allow subpatterns to be embedded inside parent patterns and processed seamlessly by the Pattern's `next()` method.  Patterns that support embedding have a 'period' argument that regulate how many items are read from it before a parent pattern can move to its next item. The period value can be expressed as a constant value or a pattern of values.

We will take a first look at these concepts using the simplest musx pattern, the Cycle.

<a id='cycle_section'></a>
## The Cycle pattern

`Cycle(items, period=None)`

A Cycle pattern generates items in an endless loop from front to back. To create a cyclic patter call Cycle and pass it a list of things you want it to produce:

In [ ]:
pat = Cycle(['a','b','c','d'])
print(f"pat: {pat}")

### Reading pattern data

To read data from a pattern call its `next()` method, which supports additional features not available in python's builtin `next()` function.

`Pattern.next(more=False)`

Pattern's `next()` method allows pattern items to be accessed in several different ways. If the *more* argument is False (the default), then `next()` returns just the next item from the pattern:

In [ ]:
pat = Cycle(['a','b','c','d'])
print(f"next: {pat.next()}")
print(f"next: {pat.next()}")
print(f"next: {pat.next()}")
print(f"next: {pat.next()}")

If the *more* argument is an integer then `next()` returns that many elements from the pattern. In the next example the cyclic ordering 1, 2, 3, 4, 1... is preserved in the output no matter how many items are returned in each call. The length of values returned is called the *period* of the pattern:

In [ ]:
pat = Cycle(['a','b','c', 'd'])
print(f"pat: {pat}")
print(f"next: {pat.next(7)}")
print(f"next: {pat.next(4)}")
print(f"next: {pat.next(1)}")
print(f"next: {pat.next(5)}")

### Pattern periods

Musx patterns allow a period length to be specified to a pattern when it is created. If the period length is not provided by the caller a default period length will be set according to the specific Pattern being used.

In this next example, the period becomes something that the pattern controls internally. To tell the pattern to use its internal period length True to the *more* parameter, as seen in this example:

In [ ]:
pat = Cycle(['a','b','c', 'd'], period=3)
print(f"pat: {pat}")
print(f"next: {pat.next(True)}")
print(f"next: {pat.next(True)}")
print(f"next: {pat.next(True)}")
print(f"next: {pat.next(True)}")

Note that True actually returns the *remaining* items in a period; this keeps ordering in sync no matter what you pass to next()'s *more* argument when you call it.

In [ ]:
pat = Cycle(['a','b','c', 'd'], period=4)
print(f"pat: {pat}")
print(f"reading remaining items in period: {pat.next(True)}")
print(f"reading 1 item: {pat.next()}")
print(f"reading remaining items in period: {pat.next(True)}")
print(f"reading 3 items: {pat.next(3)}")
print(f"reading remaining items in period: {pat.next(True)}")
print(f"reading remaining items in period: {pat.next(True)}")

The period of a pattern can also be specified as a _pattern_ of integers, in which case the pattern's period will vary accordingly. In this next example, the length of the periods will cycle through the lengths: 5, 6, 7, 5...

In [ ]:
pat = Cycle(['a','b','c', 'd'], period=Cycle([5,6,7]))
print(f"pat: {pat}")
for _ in range(7):
    print(f"next: {pat.next(True)}")

### Embedded subpatterns

The period length is particularly important when working with *embedded* patterns, i.e. subpatterns. In this example, an outer cyclic pattern contains an embedded cyclic pattern. The outer pattern cycles through four items: [10, *subpattern*, 20, 30], the subpattern cycles through two items: [-1, -2].  

In [ ]:
pat = Cycle(['a', 
             Cycle([-1, -2], period=Cycle([1,2,3])), 
             'b', 
             'c'])
print(f"{pat.next(20)}")

Note that each time the subpattern is encountered it yields 1, 2 or 3 of its items before the outer pattern can move on to its next item. The overall pattern is a merge of two streams of cyclic data: an outer pattern of 'a, 'b, 'c...
and an inner pattern -1, -2... In this image the yields from each pattern are marked by '**'.
```
outer:  **       **   **   **           **   **   **               **   **   **       **   **   **
       ['a', -1, 'b', 'c', 'a', -2, -1, 'b', 'c', 'a', -2, -1, -2, 'b', 'c', 'a', -1, 'b', 'c', 'a']
inner:       **                 **  **                 **  **  **                 ** 
```

### Embedded expressions

You can also embed python expressions inside patterns to calculate new values to return each time `next()` encounters them in the pattern. To embed an expression wrap it in a 'thunk', i.e. a lambda expression or function of zero arguments, and add that to the pattern's list of items.

In this example the constants 'a' 'b' and 'c' are interleaved with two lambda expressions:

In [ ]:
items = ['a', lambda: between(10, 20), 'b', lambda: between(-20, -10), 'c']

print(f"data: {Cycle(items).next(20)}")

### The basic rule for patterns:

For most musx patterns the basic rule to remember is: constant data can usually be replaced by patterns or expressions of data, and embedded to any level in (other) patterns.

<a id=palindrome_section></a>
## The Palindrome pattern

`palindrome(items, period=None, wrap='++')`

The Palindrome visits items first forwards and then backwards. The `wrap` argument is a two character token that determines if the first or last elements are directly repeated when the pattern reverses: the first token applies to the first item and the second item applies to the second.  The '+' token means the element is repeated and '-' means it is not repeated.

This palindrome uses the default wrap value ('++') which means that both the front and back items repeat on reversal:

In [ ]:
pat = Palindrome(['a','b','c'])
for _ in range(3):
    print(f"pat: {pat.next(True)}")

This Palindrome repeats only the front item on reversal:

In [ ]:
pat = Palindrome(['a','b','c'], wrap='+-')
for _ in range(3):
    print(f"pat: {pat.next(True)}")

This palindrome repeats only the back item on reversal:

In [ ]:
pat = Palindrome(['a','b','c'], wrap='-+')
for _ in range(3):
    print(f"pat: {pat.next(True)}")

If wrap includes only '-' signs the palindrome does not repeat either the front or back item on reversal:

In [ ]:
pat = Palindrome(['a','b','c'], wrap='--')
for _ in range(3):
    print(f"pat: {pat.next(True)}")

<a id='range_section'></a>
## The Range pattern
`Range(stop)` `Range(start, stop)` `Range(start, stop, step)` `Range(start, stop, step, period)`

Range is similar in syntax and functionality to python's range() generator but there are important differences:

- The Range pattern does not terminate: once a Range moves to its boundary it re-initializes itself to the next start, stop and step values as determined by the parameter arguments passed in.
- The start, stop and step parameters for a Range will accept integers, patterns, or thunks (lambda expressions or functions of zero arguments). 
- Since patterns always return something, an incompatible start, stop or step specifications will raise an error rather than return nothing, as Python's range() does.
- The default period length will be the number of steps between start and stop. For custom period lengths pass the period parameter an integer, pattern or thunk.

Range with a default period length of 9:

In [ ]:
r = Range(1,10)
print(f"{r.next(True)}")
print(f"{r.next()}")
print(f"{r.next(5)}")
print(f"{r.next(True)}")

Descending Range whose start and step values are patterns:

In [ ]:
pat = Range(Cycle([60, 74]), 48, Cycle([-1,-2,-3]))
print(f"pat 1: {pat.next(True)}")
print(f"pat 2: {pat.next(True)}")
print(f"pat 3: {pat.next(True)}")

Range with a cyclical period:

In [ ]:
pat = Range(0, 10, 1, period=Cycle([2,3,4]))
print(f"pat 1: {pat.next(True)}")
print(f"pat 2: {pat.next(True)}")
print(f"pat 3: {pat.next(True)}")
print(f"pat 3: {pat.next(True)}")

Play the range:

In [ ]:
pat = Range(Cycle([60, 74]), 48, Cycle([-1,-2,-3]))

def playrange(score, pat, reps, rate):
    r, d = 0, rate/2
    while r < reps:   
        keys = pat.next(True)
        t = score.now
        for k in keys:
            score.add( Note(time=t, duration=d, pitch=k, amplitude=.7))
            t += rate
        r += 1
        yield rate * len(keys)

# Create a score and give it a seq to hold the score event data.
score = Score(out=Seq())
score.compose(playrange(score, pat, 10, .125))
# Write the tracks to a midi file in the current directory.
file = MidiFile("playrange.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

<a id='shuffle_section'></a>
## The Shuffle pattern

`Shuffle(items, period=None, norep=False`

Shuffle returns random permutations of its items.  To disallow immediate repetition of an item after reshuffling (e.g. the previous choice is also the next choice) specify True to the *norep* parameter.

In this example *norep* is False, so an item can repeat after every three values:

In [ ]:
data = Shuffle(['a','b','c']).next(21)
print(f"data: {data}")

If the *norep* parameter is True, the pattern rejects shuffles that would repeat the last item as the next item. 

In [ ]:
data = Shuffle(['a','b','c'], norep=True).next(21)
print(f"data: {data}")

Musical example of Shuffle patterns that generate a series of jazz sonorities each of which is made up of a mix of 3, 4, or 5 notes from a dorian scale.

In [ ]:
from musx import keynum, intempo, Note, Seq, Score, MidiFile, playfile

def playjazzriffs(score, num, amp):
    keypat = Cycle([Shuffle(keynum("c6 d ef f g a bf"), Choose([3, 4, 5])),
                    Shuffle(keynum("c5 d ef f g a bf"), Choose([3, 4, 5])),
                    Shuffle(keynum("c4 d ef f g a bf"), Choose([3, 4, 5]))
                    ], period=1)
    rhypat = Cycle([1, 1, 1, .5])
    for _ in range(num):
        rhy = intempo(rhypat.next(), 160)
        chd = keypat.next(True)
        score.add( Note(time=score.now, duration=rhy, pitch=chd, amplitude=amp))
        yield rhy

# Create a score and give it a seq to hold the score event data.
score = Score(out=Seq())
score.compose(playjazzriffs(score, 30, .8))
# Write the tracks to a midi file in the current directory.
file = MidiFile("jazzriff.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

<a id='choose_section'></a>
## The Choose pattern

`Choose(items, weights=[], Period=None)`

Choose generates items based on weighted random selection. If a *weights* argument is not provided all items will have an equal probability of being selected. Each weight can be expressed as an integer, float, or 'thunk' -- a lambda expression or function of zero arguments. A thunk produces weight values *dynamically*, and is evaluated just before the next period of the pattern starts.

Support for displaying histograms of 1000 Choose values. If matplotlib is available a graphic image as well a histogram dictionary will be presented:

In [ ]:
try: 
    import matplotlib.pyplot as plt
    plot = True
except:
    plot = False

print(f"Matplotlib {'is' if plot else 'not'} installed, results will {'include' if plot else 'omit'} a graphic image :{')' if plot else '('}")
    
def hist (data):
    d = dict.fromkeys(sorted(data))
    for i in d:
        d[i] = data.count(i)
    print(f"histogram: {d}")
    if plot:
        plt.hist(data)

Histogram of equal probabilities:

In [ ]:
data = Choose(['A','B','C']).next(1000)
hist(data)

Histogram of 1000 datapoints yields weighted probabilities where the value 'B' is four as likely as 'A' and value 'C' is two times as likely:

In [ ]:
data = Choose(['A','B','C'], [1,4,2]).next(1000)
print(hist(data))

In this next Choose pattern the dynamic probability weight of item 'b' increases from 0 to 10 over 50 selections using linear interpolation:

In [ ]:
end = 50          # number of items to generate
res = []          # list of results
wei = 0           # current weight of 'b' item
dyn = lambda:wei  # b's dynamic weight (a closure over wei variable)

pat = Choose(['a', 'b', 'c'], [1, dyn, 1], period=1)

for i in range(end):
    wei = interp(i, [0, 0, end, 10])
    res.append(pat.next())
    
print(f"results: {res}")

A four-voice musical hommage to John Cage using dynamic weights that gradually foreground the notes C-A-G-E out of a G-Dorian background:

In [ ]:
def john_cage(score, num, trans):
    w = 0
    envl = [0, .5, 1, 6]
    pat1 = Choose(keynum('g3 a bf c4  d  e  f g'),
                         [lambda: w, # G
                          lambda: w, # A
                          1,         # Bb
                          lambda: w, # C
                          1,         # D
                          lambda: w, # E
                          1,         # F
                          lambda:w], # G
                  period=1)
    # rhythms are equal probability of quarter, half, or two eights.
    pat2 = Choose([1/4, 1/2, Cycle([1/8], period=2)]) 
    for i in range(num):
        w = interp(i/num, envl)
        k = pat1.next() + trans # add transposition offset
        r = intempo(pat2.next(), 60)
        score.add( Note(time=score.now, duration=r, pitch=k, amplitude=.7))
        yield r
        
# Create a score and give it a seq to hold the score event data.
score = Score(out=Seq())
parts = [john_cage(score, 100, trans) for trans in [-12, 0, 12, 24]]
score.compose( parts )
# Write the tracks to a midi file in the current directory.
file = MidiFile("cage.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

<a id='graph_section'></a>
## The Graph Pattern

`Graph(nodes, period=None)`

A Graph is a network of nodes, each node is a 2- or 3-tuple containing an *item*, *link* and optional *id*: 

`python
(item, link, [id])
`
The *item* is the value to return from the node. The node's *link* is the identifier of the next node to visit, and *id* is the node's own unique identifier in the graph.

The values for *item* and *link* can be constants, subpatterns, or thunks. If an optional *id* is not specified then that node's identifier will be automatically set to the node's 1-based position in the list of nodes specified to the pattern.

This example represents an alberti bass figure as a graph. The top lines shows the generated ids for each of the three nodes. The graph starts with the first node (G4) which then moves to node 3 (G4) which moves to node 2 (E), which moves to node 1 (C), and so the pattern starts over:


In [ ]:
# node ids: 1          2          3
g = Graph([('C4', 3), ('E4', 3), ('G4', Cycle([2, 1], period=1))], period=4)
print(f"period: {g.next(True)}")
print(f"period: {g.next(True)}")
print(f"period: {g.next(True)}")

This example uses a graph to generate a mostly stepwise diatonic melody from its given scale (C-Dorian). Each segment of the melody ends with a rest (the 'r' item):

In [ ]:
melody = Cycle([Graph([("c4",  Choose([2, 5])),
                       ("d4",  Choose([1, 3])), 
                       ("ef4", Choose([2, 4])),
                       ("f4",  Choose([3, 5])),
                       (Shuffle(["g4", "a4", "bf4", "c5"]), Cycle([1, 2, 3, 4]))
                       ]),
                'r'
                ])

def playgraph(score, segs, mel, rate):
    # plays segs number of segments, each segment ends with an "r" (rest).
    n = 0
    while n < segs:
        p = mel.next()
        if p != 'r':
           score.add(Note(time=score.now, duration=rate*1.5, pitch=keynum(p), amplitude=.7))
        else:
           n += 1
        yield rate      

score = Score(out=Seq())
score.compose( playgraph(score, 10, melody, .25) )
# Write the tracks to a midi file in the current directory.
file = MidiFile("graph.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

<a id='rotation_section'></a>
## The Rotation pattern

`Rotation(items, rules, period=None)`

Rotation permutes it's items according to one or more *swapping rules*. A swapping rule is a list of (up to) four integers that defines how elements are swapped (exchanged) in order to produce the next arrangement, or *generation*, of data:

<code>    [start, step, span=1, end=len]</code>

*Start* is the location (zero based) in the items list to begin swapping. *Step* is the rightward increment to move to start the next swap. *Span* is the optional distance between the two elements that will be swapped (default is adjacent elements). *End* is the optional index in the data that stepping stops before (default is length of the data).

Here is an example of rotating a list of three elements [a, b, c] using one rule [0, 1, 1] until the original ordering returns. The sequential swaps that transform the current ordering (generation) into the next are shown in bold.

| this generation  | → swap #1 | → swap #2 | → next generation |
|:------------|:--------|:-------|:---------|
| a b c |  **b a** c | b **c a**  | b c a |
| b c a	|  **c b** a	| c **a b**  | c a b |
| c a b	|  **a c** b	| a **b c**  | a b c |

To specify multiple rules, provide a list of rules or a generator of rules, in which case the next rule is used to produce the next generation of data.

In this example adjacent pairs of items are swapped

In [ ]:
data = ['a','b','c']
rule = [0, 1, 1]
rgen = Rotation(data, rule)
for g in range(4):
    print(f"generation{g+1}: {rgen.next(True)}")

#### `Rotation.all(grouped=False, wrapped=False)`

Rotation provides the helper function `all()` that reads all the generations of its data until the original generation returns. If the *grouped* parameter is False then the generations are returned as a flat list of elements, if it is True then each generation is returned in its own sublist. If *wrapped* is True then the terminating reappearance of the first generation will be included in the data returned.

This example  uses two rules: the first rules swaps just elements 0 and 1, and the second rule swaps elements 1 and 2. Since grouped and wrapped are both True each generation is returns as a sublist and the last generation is the same as the first:

In [ ]:
data = ['a','b','c']
rules = [[0, 1, 1, 2], [1, 1, 1, 3]]

rpat = Rotation(data, rules)

for g, gen in enumerate(rpat.all(True, True)):
    print(f"gen{g+1}: {gen}")

This next example returns the same results but as a flat list without the inititial generation at the end of the results:

In [ ]:
data = ['a','b','c']
rules = [[0, 1, 1, 2], [1, 1, 1, 3]]
rpat = Rotation(data, rules)
print(f"all no repeats: {rpat.all(False, False)}")

#### changeringing.ipynb

For many musical examples using the Rotation pattern see the musx tutorial [changeringing.ipynb](changeringing.ipynb) in the same directory as this file.

<a id='markov_section'></a>
## The Markov pattern

`Markov(rules, stop=None, initial=[])`

### Background

A *markov process* (markov chain) is a random process whose outcomes are affected by previous choices :

<img src="support/markov1.jpg">

We can think of this process as a table where each row defines the outcome probabilities of one (or more) past choices. Outcomes from the process are then fed back into the process to determine the next outputs.
    
<img src="support/markov2.jpg">

* Line one: The probablity of A given A is .5 and the probability of B given A is .5.
* Line two: The probability of A given B is .2 and the probability of B given B is .8.
    
<p>
A Markov process can be easily represented as a <I>transition table</I> that maps inputs to outputs according to probabilities.  The total probability of each transition (row) must equal 1.0, meaning that some outcome in a row must result from the past choice(s).
</p>

<img support="support/markov3.jpg">

<p>A Markov process is generic -- its outputs can be applied to many types of data sets. In music the data might be pitches, rhythms, harmonic choices, etc.  Because past choices influence subsequent choices the process can <I>mimic</I> many types of musical pattern behavior.</p>

<img support="support/markov4.jpg">
    

### The transition table

The Markov pattern produces its items using a Python dictionary of (nth order) transition rules whose keys and values link past outcomes to a set of possible future outcomes. The dictionary's keys holds one or more past value(s), and the key's value is a list that associates each possible outcome with its probability:

Dictionary syntax:

<pre>
{
 (past1, ...): [[next1, prob], [next2, prob], ...], 
 (past2, ...): [[next1, prob], [next2, prob], ...],
 ...
}
</pre>

As an example, here is a first order Markov process that outputs a pattern of 'a', 'b', and 'c'. The first rule says that if the last outcome was 'a' then the next outcome will be 'b' or 'c', with 'c' three times a likely to be chosen as 'b'.  The second rule says that if the previous output is 'b' then 'a' always follows. The third rules say that if 'c' was the last outcome, then any of the letters can result but 'a' is likely than 'b' or 'c' and 'c' is more likely than 'a'.

This example a Markov pattern, note that each time 'b' occurs 'a' follows immediately:

In [ ]:
rules = {('a'): [['b', 1], ['c', 3]], 
         ('b'): [['a', 1]], 
         ('c'): [['a', 5], ['c', 1], ['b', 2.5]]}
pat = Markov(rules)
data = pat.next(20)
print(data)

For convenience sake, in the case of first order Markov where rules only use one past event (see example above) the rule's key can be the item itself, rather than a tuple containing that item.  In addition, any future value that would otherwise be a list with a weight of 1 can be replaced by just the future value.  

This next example uses both these conveniences to make a simplified version of the rules.

In [ ]:
rules = {'a': ['b', ['c', 3]], 
         'b': ['a'], 
         'c': [['a', 5], 'c', ['b', 2.5]]}
pat = Markov(rules)
data = pat.next(20)
print(data)

### Markov orders and `Markov.analyze()`

One of the most interesting uses of Markov is to model real-world phenomena by analyzing data to extract the probability of outcomes given previous outcomes. Using such an analysis one can then determine rules that will generate output approaching the original data. How similar the results will depend on "N", the number of previous outcomes that are used to determine the next outcome. The number N is called the *markov order* of the process, and as the order increases, a Markov process will come closer and closer to exactly mimicking the data it is derived from.

The next examples demonstrate the effect of increasing the order of a Markov process to create melodies that come closer and closer to the original data.

Markov setup:

In [ ]:
# The melodic pitches to use
melody = [60, 60, 62, 60, 65, 64, 60, 60, 62, 60, 67, 65,
          60, 60, 72, 69, 65, 64, 62, 70, 70, 69, 65, 67, 65]
        
def playmarkov(mpat):
    '''Generates output from a markov pattern to a midi file.'''
    def markovmelody(score, reps, mpat, rate):
        '''Generates notes to a score from a markov pattern.'''
        for _ in range(reps):
            p = mpat.next()
            score.add(Note(time=score.now, pitch=p, duration=rate * 1.5))
            yield rate
    print(f"rules: {mpat.items}")
    seq=Seq()
    score=Score(out=seq)
    score.compose( markovmelody(score, 50, mpat, .2) )
    f=MidiFile("markovmelody.mid", seq).write()
    print(f"Wrote '{f.pathname}'")
    playfile(f.pathname)
    
print(f'playmarkov: {playmarkov}')

### Markov.analyze(data, order)

The following examples call the `Markov.analyze()` factory method to return patterns with increasing orders that come closer and closer to the original melody.  At what order do you recognize the tune?

1st order Markov considers one past value, the original melody is only vaguely present:

In [ ]:
markov = Markov.analyze(melody, order=1)
playmarkov(markov)

2nd order Markov considers two past values, some motives are recognizable:

In [ ]:
rules = Markov.analyze(melody, order=2)
playmarkov(rules)

3rd order Markov...getting closer!

In [ ]:
rules = Markov.analyze(melody, order=3)
playmarkov(rules)

4th order Markov, you should recognize the melody by now:

In [ ]:
rules = Markov.analyze(melody, order=4)
playmarkov(rules)

In 5th order Markov the pattern is completely determined since there is only one possible output for each row:

In [ ]:
rules = Markov.analyze(melody, order=5)
playmarkov(rules)

### Markov Examples

#### Markov Chanting

In the crudest of terms, a Gregorian Chant is characterized by mostly stepwise motion within a range of modal degrees. From any given tone there is more likelihood that a chant will move a step up or down than leap to a tone further away. The larger the skip, the more unlikely it is to occur. In addition, certain tones, such as the
final and tenor, have more influence over the melody than other tones. For example, in the Dorian mode the tenor note A is occasionally decorated by the B-flat directly above it. This B-flat almost always returns back to the tenor tone. In an authentic mode, the final of the mode also acts as a kind of reflecting boundary
that redirects the melody in the opposite direction. 

We can loosely mimic these stylistic tendencies using a first order Markov process. (The table values were created ad hoc, a much better way would be to analyze a collection of dorian chants...):

In [ ]:
chantrules = { 
    ('d4',): [['d4', .1],  ['e4', .35], ['f4', .25], ['g4', .1], ['a4', .15]],
    ('e4',): [['d4', .35], ['e4', .1], ['f4', .35], ['g4', .1], ['a4', 1]],
    ('f4',): [['d4', .2], ['e4', .2], ['f4', .1], ['g4', .2], ['a4', .12]],
    ('g4',): [['d4', .2], ['e4', .1], ['f4', .3], ['g4', .1], ['a4', .3], ['bf4', .2]],
    ('a4',): [['d4', .1], ['e4', .1], ['f4', .2], ['g4', .3], ['a4', .2], ['bf4', .3]],
    ('bf4',): [['a4', 1]]
}

def monk(score, reps, chant, rhy):
    pat = Markov(chant)
    for _ in range(reps):
        k = keynum(pat.next())
        n = Note(time=score.now, pitch=k, duration=rhy)
        score.add(n)
        yield rhy
        
def playchant(rules):
    m=MidiFile.metatrack(ins={0: ChoirAahs})
    t=Seq()
    s=Score(out=t)
    s.compose( monk(s, 50, chantrules, .35) )
    f=MidiFile("chant.mid", [m, t]).write()
    playfile(f.pathname)
    return rules

print(f"OK!")

In [ ]:
playchant(chantrules)

In the next example we can add some rhythmic interest by possibly adjusting a note's duration if it is in the tonic triad and also forcing the last note to be the tonic.

In [ ]:
def chant_dur(knum, dur):
    pc = knum % 12
    if pc == 2:
        return odds(.7, dur * 2, dur)
    elif pc == 9:
        return odds(.5, dur * 2, dur)
    elif pc == 5:
        return odds(.25, dur * 2, dur)
    return dur

def choir_monk(score, endtime, chant, dur, octave):
    pat = Markov(chant)
    while True:
        k = keynum(pat.next())
        if score.elapsed > endtime and ((k % 12) == 2):
            yield -1
        d = chant_dur(k, dur)
        score.add(Note(time=score.now, pitch=k+octave, amplitude=.8, duration=dur))
        yield dur
print(f'choir_monk: {choir_monk}')

For fun will run three monks simultaneously at the interval of a tritone.

In [ ]:
t=Seq()
m=MidiFile.metatrack(ins={0: ChoirAahs})
s=Score(out=t)
s.compose([choir_monk(s, 20, chantrules, .8, 6), 
           choir_monk(s, 20, chantrules, .8, 0) ,  
           choir_monk(s, 20, chantrules, .8, -6)])
f=MidiFile("chorus.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

#### Markov Harmony

A Markov process can be used to determine an interesting mix intervals that can be stacked to create different chords.
This example uses an ad hoc table to compose harmony and melody vaguely reminiscent of Messiaen's modes.

In [ ]:
harmonyrules = {1: [[3, .4], [4, .4], [6, .1]],
                2: [[2, .2], [3, .4], [4, .4], [6, .1]],
                3: [[1, .2], [2, .6], [4, .4]],
                4: [[2, .2], [3, .4], [4, .4]],
                6: [[2, .4], [3, .2], [4, .2]]
}

def markov_harmonizer (score, length, rules, top, size, upward, rhy, dur):
    pat = Markov(rules)
    for _ in range(length):
        i = pat.next()
        top = fit(top + odds(upward, i, -i), 50, 90)
        k = top
        for _ in range(size):
            m = Note(time=score.now, pitch=k, duration=dur)
            score.add(m)
            k -= pat.next()
        yield rhy

t=Seq()
m=MidiFile.metatrack(ins={0: AcousticGrandPiano}) 
s=Score(out=t)
s.compose(markov_harmonizer(s, 25, harmonyrules, 67, 6, .7, 1.2, 1.2))
f=MidiFile("harmony.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

#### Markov Rhythmic Patterns

Imagine generating a melody for a soloist in which the rhythms are determined by weighted random selection. Even if only a few simple rhythms are used, the 'metric pulse' of the process will be weak since sixteenths and dotted eighths can appear anywhere, only occasionally lining up with the start of a metric pulse. If a tempo curve is also applied the beat becomes even more obscured.

In [ ]:
def ranrhythm(s, leng, rhys, lo, hi):
    tcurve = [0, 1, .7, .75, 1, 1]
    for i in range(leng):
        k = between(lo, hi)
        r = rhys.next()
        d = intempo(r, 120) * interp(i / leng, tcurve)
        m = Note(time=s.now, pitch=k, duration=d)
        s.add(m)
        yield d

# rhythms are random
rhythms = Choose([1, .75, .5, .25])
        
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(ranrhythm(s, 75, rhythms, 40, 80))
f=MidiFile("rhythm.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

In the next version a first order Markov process produces random variations that support the perception of an underlying pulse, even in the case of a tempo curve warping the timing. Table labels are Q=quarter, E.=dotted eighth, E=eighth, S=sixteenth, and table values are probability weights.

<table>
    <tr> <td> </td>   <td>Q</td>   <td>E.</td>   <td>E</td>  <td>S</td>  </tr>
    <tr> <td>Q</td>   <td>.5</td>  <td>.75</td>  <td>2</td>  <td>2</td>  </tr>
    <tr> <td>E.</td>  <td>0</td>   <td>0</td>    <td>0</td>  <td>1</td>  </tr>
    <tr> <td>E</td>   <td>2</td>   <td>1</td>    <td>2</td>  <td>0</td>  </tr> 
    <tr> <td>S</td>   <td>1</td>   <td>0</td>    <td>2</td>  <td>1</td>  </tr> 
    <tfoot></tfoot>
</table>

In [ ]:
rules = {
    1: [[1, .5], [.75, .75], [.5, 2]],
    .75: [.25],
    .5: [[1, 2], .75, [.5, 2]],
    .25: [1, [.5, 2], .25]
}

# ryhthms are 
rhythms = Markov(rules)

t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(ranrhythm(s, 75, rhythms, 40, 80))
f=MidiFile("rhythm.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

#### Markov Texture

This example is a take on keyboard motions between black keys and white keys that can be found in some of Ligeti's works. THe choice of octaves is also made using a Markov process.

In [ ]:
bwmotion = {
    0:  [[0, .5],  [2, 2],   [4, 1.5], [5, 1],   [7, .5],   [9, .5],  [11, .5],  [1, .2], [3, .1], [6, .1], [8, .1],  [10, .1]],
    1:  [[1, .5],  [3, 2],   [6, 1.5], [8, 1],   [10, 1],   [0, .2],  [2, .2],   [4, .1], [5, .1], [7, .1], [9, .1],  [11, .1]],
    2:  [[0, 2],   [2, .5],  [4, 2],   [5, 1.5], [7, 1],    [9, .5],  [1, .2],   [3, .2], [6, .1], [8, .1], [10, .1], [11, .1]],
    3:  [[1, 2],   [3, .5],  [6, 1.5], [8, 1],   [10, .5],  [0, .1],  [2, .2],   [4, .2], [5, .1], [7, .1], [9, .1],  [11, .1]],
    4:  [[0, 1.5], [2, 2],   [4, .5],  [5, 2],   [7, 1.5],  [9, 1],   [11, .5],  [1, .1], [3, .2], [6, .2], [8, .1],  [10, .1]],
    5:  [[0, 1],   [2, 1.5], [4, 2],   [5, .5],  [7, 2],    [9, 1.5], [11, 1],   [1, .1], [3, .2], [6, .2], [8, .1],  [10, .1]],
    6:  [[1, 1.5], [3, 2],   [6, .5],  [8, 2],   [10, 1.5], [0, .1],  [2, .1],   [4, .1], [5, .2], [7, .2], [9, .1],  [11, .1]],
    7:  [[0, .5],  [2, 1],   [4, 1.5], [5, 2],   [7, .5],   [9, 2],   [11, 1.5], [1, .1], [3, .1], [6, .2], [8, .2],  [10, .1]],
    8:  [[1, 1],   [3, 1.5], [6, 2],   [8, .5],  [10, 2],   [0, .1],  [2, .1],   [4, .1], [5, .1], [7, .2], [9, .2],  [11, .1]],
    9:  [[0, .5],  [2, .4],  [4, 1],   [5, 1.5], [7, 2],    [9, .5],  [11, 2],   [1, .1], [3, .1], [6, .1], [8, .2],  [10, .2]],
    10: [[1, .5],  [3, 1],   [6, 1.5], [8, 2],   [10, .5],  [0, .1],  [2, .1],   [4, .1], [5, .1], [7, .1], [9, .2],  [11, .2]],
    11: [[0, .5],  [2, .5],  [4, .5],  [5, 1],   [7, 1.5],  [9, 2],   [11, .5],  [1, .1], [3, .1], [6, .1], [8, .1],  [10, .2]]
}

bwoctaves = {
    'c3': [['c3', 2],   ['c4', 1],  ['c5', .5], ['c6', .25]],
    'c4': [['c3', 1],   ['c4', 2],  ['c5', 1],  ['c6', .5]],
    'c5': [['c3', .5],  ['c4', 1],  ['c5', 2],  ['c6', 1]],
    'c6': [['c3', .25], ['c4', .5], ['c5', 1],  ['c6', 2]]
}

def bwkeys(score, length, octlist, intlist, rate, chan):
    ints = Markov(intlist)
    octs = Markov(octlist)
    reps = 0
    octv = 0
    for _ in range(length):
        if reps == 0:
            reps = pick(4, 8, 12, 16)
            octv = keynum(octs.next())
        intr = ints.next()
        n = Note(time=score.now, duration=rate * 1, pitch=octv + intr, instrument=chan)
        score.add(n)
        reps = reps - 1
        yield rate

print(f'bwkeys: {bwkeys}')

Listen to one voice:

In [ ]:
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(bwkeys(s, 120, bwoctaves, bwmotion, .125, 0))
f=MidiFile("texture.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

Listen to two voices:

In [ ]:
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba, 1: Marimba})
s=Score(out=t)
s.compose([[0, bwkeys(s, 120, bwoctaves, bwmotion, .125, 0)],
          [2, bwkeys(s, 120, bwoctaves, bwmotion, .125, 1)]])
f=MidiFile("texture.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

For a larger example of a musical Markov pattern see the demo file foster.ipynb.

<a id='states_section'></a>
## The States pattern

`States(cells, transfunc)`

States returns evolving values from a state machine (cellular automata). The pattern supports both one and two dimensional automata; for a 1D automata the _cells_ argument is a list of states, for 2D automata it is a list of lists where each sublist defines a 'row' of initial states. 

### The transition function and `States.getstate()`

States applies a user-defined _transition function_ to the automata's cells to produce the next generation of states. This function accepts two arguments, the cells array and an index, and returns the next state for the cell at the given index:

    transfunc(cells, index)

To determine the next state of the indexed cell the transition function typically has to access values in neighboring cells. The States class provides a static function that will access the current state of any cell in the array using relative offsets _xinc_ and _yinc_  to the current index:

    States.getstate(cells, index, xinc=0, yinc=0)
    
For example, in a 1D automata: <pre>States.getstates(cells, index, 1)</pre> would return the state value of the cell to the immediate right of the indexed cell, and: <pre>States.getstate(cells, index, -2)</pre> would return the state of the cell two positions to the left of the current cell. The States.getstate() function computes addresses mod the size of the cell array so the transition function does not have to perform bounds checking when access neighbors. 

In this example the next state of the current cell will be determined by accessing the values of neighbor cells to the immediate left and right and adding them together mod 4:

In [ ]:
def add_neighbors(cells, index):
    left = States.getstate(cells, index, -1)
    right = States.getstate(cells, index, 1)
    #print(f"cells: {cells}, index={index}, left: {left} right: {right}")
    return (left + right) % 4

cells = States([0,1,0,1,0], transitions=add_neighbors)

for _ in range(10): 
    print(cells.next(5))

Use the pattern as decoration on an ascending line:

In [ ]:
cells = States([0,1,0,1,0], transitions=add_neighbors)
line = Range(60, 85, Cycle([1,2,3,4,5], period=3))

def playcells(score, cells, reps, line, rate, amp):
    for i in range(reps):
        if i % 5 == 0:
            k = line.next()
        n = Note(time=score.now, pitch=cells.next() + k, duration=rate+.1, amplitude=amp)
        score.add(n)
        yield rate

score=Score(out=Seq())
score.compose(playcells(score, cells, 120, line, .125, .7))
file=MidiFile("states.mid", [score.out]).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)

This example sonifies an eight-state automata. The values from the pattern controls both pitch and rhythm:

In [ ]:
def eight_states (cells, index):
    logand = lambda x,y: x & y   
    left = States.getstate(cells, index, -1)
    middle = States.getstate(cells, index, 0)
    right = States.getstate(cells, index, 1)
    return logand(left, 4) + logand(middle, 2) + logand(right, 1)

cells = States([0, 1, 2, 3, 4, 5, 6, 7], eight_states)

def play_eight_notes(score, cells, reps, amp):
    keys = keynum("c4 fs af b d5 ef e g")
    for _ in range(reps):
        i = cells.next()
        k = keys[i]
        r = rescale(i, 0, 7, .150, .400)
        n = Note(time=score.now, duration=r*1.9, pitch=k, amplitude=amp)
        score.add(n)
        yield r

score=Score(out=Seq())
score.compose(play_eight_notes(score, cells, 128, .7))
file=MidiFile("states.mid", [score.out]).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)